# 5.SQL and Dataframes

References:

* Spark-SQL, <https://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes>


# 5.1  Example Walkthrough
Follow the Spark SQL and Dataframes Examples below!

### Initialize PySpark

First, we use the findspark package to initialize PySpark.

In [1]:
import os, sys
os.chdir(os.path.join(os.environ["HOME"], "exercise-students-2020/05_SQL"))
%pwd

'/home/jupyter-ter-akopyan-118531-8d8bf/exercise-students-2020/05_SQL'

In [2]:
# Initialize PySpark
import os, sys
APP_NAME = "PySpark Lecture"
SPARK_MASTER="local[1]"
import pyspark
import pyspark.sql
from pyspark.sql import Row
conf=pyspark.SparkConf()
conf=pyspark.SparkConf().setAppName(APP_NAME).set("spark.local.dir", os.path.join(os.getcwd(), "tmp"))
sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated...")

PySpark initiated...


### Hello, World!

Loading data, mapping it and collecting the records into RAM...

In [3]:
# Load the text file using the SparkContext
csv_lines = sc.textFile("../data/example.csv")

# Map the data to split the lines into a list
data = csv_lines.map(lambda line: line.split(","))

# Collect the dataset into local RAM
data.collect()

[['Russell Jurney', 'Relato', 'CEO'],
 ['Florian Liebert', 'Mesosphere', 'CEO'],
 ['Don Brown', 'Rocana', 'CIO'],
 ['Steve Jobs', 'Apple', 'CEO'],
 ['Donald Trump', 'The Trump Organization', 'CEO'],
 ['Russell Jurney', 'Data Syndrome', 'Principal Consultant']]

### Creating Rows

Creating `pyspark.sql.Rows` out of your data so you can create DataFrames...

In [4]:
# Convert the CSV into a pyspark.sql.Row
def csv_to_row(line):
    parts = line.split(",")
    row = Row(
      name=parts[0],
      company=parts[1],
      title=parts[2]
    )
    return row

# Apply the function to get rows in an RDD
rows = csv_lines.map(csv_to_row)

### Creating DataFrames from RDDs

Using the `RDD.toDF()` method to create a dataframe, registering the `DataFrame` as a temporary table with Spark SQL, and counting the jobs per person using Spark SQL.

In [5]:
# Convert to a pyspark.sql.DataFrame
rows_df = rows.toDF()
rows_df.take(5)

[Row(company='Relato', name='Russell Jurney', title='CEO'),
 Row(company='Mesosphere', name='Florian Liebert', title='CEO'),
 Row(company='Rocana', name='Don Brown', title='CIO'),
 Row(company='Apple', name='Steve Jobs', title='CEO'),
 Row(company='The Trump Organization', name='Donald Trump', title='CEO')]

In [6]:
# Register the DataFrame for Spark SQL
rows_df.registerTempTable("executives")

# Generate a new DataFrame with SQL using the SparkSession
job_counts = spark.sql("""
SELECT
  name,
  COUNT(*) AS total
  FROM executives
  GROUP BY name
""")
job_counts.show()

# Go back to an RDD
job_counts.rdd.collect()

+---------------+-----+
|           name|total|
+---------------+-----+
|   Donald Trump|    1|
|Florian Liebert|    1|
|      Don Brown|    1|
| Russell Jurney|    2|
|     Steve Jobs|    1|
+---------------+-----+



[Row(name='Donald Trump', total=1),
 Row(name='Florian Liebert', total=1),
 Row(name='Don Brown', total=1),
 Row(name='Russell Jurney', total=2),
 Row(name='Steve Jobs', total=1)]

# 5.2-5.9 NASA DataSet

5.2 Create a Spark-SQL table with fields for IP/Host and Response Code from the NASA Log file! 

In [7]:
!tail -n5 /opt/data/nasa/NASA_access_log_Jul95

163.205.53.14 - - [28/Jul/1995:13:32:23 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204
tiger2.ocs.lsu.edu - - [28/Jul/1995:13:32:23 -0400] "GET /shuttle/missions/missions.html HTTP/1.0" 200 8677
199.0.2.27 - - [28/Jul/1995:13:32:23 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 200 5866
tornado.umd.edu - - [28/Jul/1995:13:32:25 -0400] "GET /shuttle/missions/sts-74/sts-74-patch-small.gif HTTP/1.0" 200 5494
alyssa.p

In [8]:
# Load the text file using the SparkContext
nasa_lines = sc.textFile("/opt/data/nasa/NASA_access_log_Jul95")

In [9]:
# Convert the CSV into a pyspark.sql.Row
def nasa_to_row(line):
    parts = line.split(" ")
    row = Row(
      host=parts[0],
      res=parts[-2]
    )
    return row

# Apply the function to get rows in an RDD
nasa_rows = nasa_lines.filter(lambda x: '- -' in x).map(nasa_to_row)

In [10]:
# Convert to a pyspark.sql.DataFrame
nasa_df = nasa_rows.toDF()

In [11]:
nasa_df.take(5)

[Row(host='199.72.81.55', res='200'),
 Row(host='unicomp6.unicomp.net', res='200'),
 Row(host='199.120.110.21', res='200'),
 Row(host='burger.letters.com', res='304'),
 Row(host='199.120.110.21', res='200')]

5.3 Run an SQL query that outputs the number of occurrences of each HTTP response code!

In [13]:
# Register the DataFrame for Spark SQL
nasa_df.registerTempTable("nasalogs")

# Generate a new DataFrame with SQL using the SparkSession
res_counts = spark.sql("""
SELECT
  res,
  COUNT(*) AS total
  FROM nasalogs
  GROUP BY res
""")
res_counts.show()

+---+-------+
|res|  total|
+---+-------+
|200|1701534|
|302|  46573|
|501|     14|
|404|  10845|
|403|     54|
|500|     62|
|304| 132627|
|400|      5|
+---+-------+



5.4 Cachen Sie den Dataframe und führen Sie dieselbe Query nochmals aus! Messen Sie die Laufzeit für das Cachen und für die Ausführungszeit der Query!

In [14]:
from time import time

In [15]:
cstart = time()
nasa_df.cache().count()
cend = time()
print('Cache runtime: %.3f' % (cend - cstart))

Cache runtime: 11.949


In [16]:
qstart = time()
res_counts = spark.sql("""
SELECT
  res,
  COUNT(*) AS total
  FROM nasalogs
  GROUP BY res
""")

res_counts.show()
qend = time()
print('Query runtime: %.3f' % (qend - qstart))

+---+-------+
|res|  total|
+---+-------+
|200|1701534|
|302|  46573|
|501|     14|
|404|  10845|
|403|     54|
|500|     62|
|304| 132627|
|400|      5|
+---+-------+

Query runtime: 1.269


5.5. Implement the same Query using the Dataframe API!

5.6 Führen Sie diesselbe Query mit/ohne Cache und 8, 16 Cores aus! Dokumentieren und erklären Sie das Ergebnis!

5.7 Performance Analysis: 
* Create RDDs with 2x, 4x, 8x and 16x of the size of the NASA log dataset! Persist the dataset in the Spark Cache! Use an appropriate number of cores (e.g. 8 or 16)!
* Measure and plot the response times for all datasets using a constant number of cores!
* Plot the results!
* Explain the results!



5.8 Strong Scaling

  * **Measure the runtime for the query for 1, 2, 4, 8 and 16 cores for 1x and 16x datasets!** Datasets cached in Memory!
  * Compute the speedup and efficiency!
  * Plot the responses!
  * Explain the results!